In [50]:
import os
import sys
import numpy as np
import csv
import pandas as pd
import pydicom
import SimpleITK as sitk
from pydicom import dcmread, FileDataset
from rt_utils import RTStructBuilder, RTStruct
import dateutil
import os
import time

In [12]:
path = './../../../../../../data/blobfuse/PSMA_PCA_LESIONS_SEGMENTATION/PET_POSITIVES/DICOM/'
patient_ids = sorted(os.listdir(path))

In [13]:
full_id_path = [os.path.join(path, patient_ids[i]) for i in range(len(patient_ids))]

In [19]:
ct_files_path = [os.path.join(full_id_path[i],'CT') for i in range(len(patient_ids))]
pt_files_path = [os.path.join(full_id_path[i],'PET') for i in range(len(patient_ids))]
rt_files_path = [os.path.join(full_id_path[i],'RT') for i in range(len(patient_ids))]

In [24]:
convert = []
for i in range(len(patient_ids)):
    convert.append('Y')

In [25]:
ctpt_convert = {'PatientID':patient_ids, 'CT_dir':ct_files_path, 'PET_dir':pt_files_path, 'convert':convert}

In [38]:
rt_convert = {'PatientID':patient_ids, 'RTSTRUCT_dir':rt_files_path, 'REF_dir':pt_files_path, 'convert':convert}

In [28]:
csv_ctpt_path = './dicom_ctpt_to_nifti_conversion_file.csv'
csv_rt_path = './dicom_rtstruct_to_nifti_conversion_file.csv'

In [34]:
with open(csv_ctpt_path, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Iterate over the rows, where each row corresponds to an element in the lists
    for row in zip(*ctpt_convert.values()):
        # Write each row to the CSV file
        writer.writerow(row)

In [39]:
with open(csv_rt_path, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)

    for row in zip(*rt_convert.values()):
        writer.writerow(row)

In [6]:
ctpt_excel_path = './dicom_ctpt_to_nifti_conversion_file.csv'

In [41]:
structs1 = pd.read_csv('/home/jhubadmin/Desktop/segmentation_research/lymphoma-segmentation-dnn/data_conversion/dicom_rtstruct_to_nifti_conversion_file.csv')
structs1 = list(structs1.itertuples(index=False, name=None))

In [63]:
struct_file = os.listdir(structs1[13][1])[0]
struct_file
# r = pydicom.read_file(os.path.join(structs1[1][1],struct_file))

'2.16.840.1.114362.1.11854208.23048987573.538891512.757.3667.dcm'

In [60]:
structs1[13][2]

'./../../../../../../data/blobfuse/PSMA_PCA_LESIONS_SEGMENTATION/PET_POSITIVES/DICOM/PSMA-01-025/PET'

In [64]:
rtstruct = RTStructBuilder.create_from(dicom_series_path=structs1[13][2], rt_struct_path=os.path.join(structs1[13][1],struct_file))

In [54]:
def get_filtered_roi_list(rois):
    filtered_rois = []
    for roi in rois:
        # if roi.endswith('PETEdge'):
        filtered_rois.append(roi)
        # else:
        #     pass
    return filtered_rois

In [ ]:
def load_merge_masks(rtstruct: RTStruct) -> np.ndarray:
    '''
    Load and merge masks from a dicom RTStruct. All of the
    masks in the RTStruct will be merged. Add an extra line
    of code if you want to filter for/out certain masks.
    '''
    rois = rtstruct.get_roi_names()
    rois = get_filtered_roi_list(rois)
    masks = []
    for roi in rois:
        print(roi)
        mask_3d = rtstruct.get_roi_mask_by_name(roi).astype(int)
        masks.append(mask_3d)

    final_mask = sum(masks)  # sums element-wise
    final_mask = np.where(final_mask>=1, 1, 0)
    # Reorient the mask to line up with the reference image
    final_mask = np.moveaxis(final_mask, [0, 1, 2], [1, 2, 0])

    return final_mask

In [65]:
rois = rtstruct.get_roi_names()

In [58]:
filtered_rois = []
for roi in rois:
    if roi.endswith('PETEdge'):
        filtered_rois.append(roi)

In [80]:
a = [1,2,3,4,5]
b = [5,6,7,8,9]

for c,d in zip(a,b):
    print(c, d)

1 5
2 6
3 7
4 8
5 9


In [59]:
filtered_rois

[]

In [79]:
# f = []
# r = sum(f)
# np.array_equal(f,0)

ctpath = '/data/blobfuse/PSMA_PCA_LESIONS_SEGMENTATION/converted_data/CT/PSMA-01-001_0000.nii.gz'

ctimg = sitk.ReadImage(ctpath)

In [49]:
final_mask = load_merge_masks(rtstruct)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 204
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: RT Structure Set Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 2.16.840.1.114362.1.11854208.23048987573.538886149.451.2447
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.45504.2.2.2.0
(0002, 0013) Implementation Version Name         SH: 'ImageDrive 2.0'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0012) Instance Creation Date              DA: '20200207'
(0008, 0013) Instance Creation Time              TM: '113028.452'
(0008, 0016) SOP Class UID                       UI: RT Structure Set Storage
(0008, 0018) SOP Instance UID                    UI: 2.16.840.1.114362.1.11854208.230